# NBA Game Dashboard Design Outline

# Game Context

- away @ home
- total / spread

# Team Context

- record / L10 record
- injuries
    - players returning from injuries 
    - starting lineups ?
    - highlight key scorers / defenders
- stats allowed (overall + dvp)
- pace(rank) ?

# Player Context

- PTS, REB, AST (+ projection)
    - FPS
    - 3s
        - REB/AST chance data

- hot / cold n-game streak
    - x out of last y
    - season hit %
    - visualize outcomes vs szn/rolling avg
        - visualize attempts/chances
- baseline deviation ?
- matchup
- stats without injured teammate
- home / away splits
- H2H performances

- Similar player recently recorded stats
- Recent scoring/assist performances vs game totals 

In [1]:
home_team = "New Orleans Pelicans"
away_team = "Los Angeles Lakers"
game_total = 235
home_spread = -6
away_spread = 6

print(f"{away_team} @ {home_team}")
print(f"Game total: {game_total}")
print(f"Spreads:\n  Away: {away_spread}\n  Home: {home_spread}")

Los Angeles Lakers @ New Orleans Pelicans
Game total: 235
Spreads:
  Away: 6
  Home: -6


In [2]:
# Connect to database
from sqlalchemy import create_engine
import pandas as pd

DB_USER = "admin"
DB_PASSWORD = "admin"
DB_HOST = "localhost"
DB_PORT = 5433
DB_NAME = "nba_db"

engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

In [3]:
query = """
ALTER TABLE boxscores.player_boxscores_traditional_v3
ADD COLUMN IF NOT EXISTS clean_name TEXT;

UPDATE boxscores.player_boxscores_traditional_v3
SET clean_name = regexp_replace(
    lower(unaccent(CONCAT_WS(' ', first_name, family_name))),
    '[^a-z ]',
    '',
    'g'
);

SELECT 
    *,
    AVG(points) OVER (
        PARTITION BY player_id
        ORDER BY game_id
        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
    ) AS ppg,
    AVG(points) OVER(
        PARTITION BY player_id
        ORDER BY game_id
        ROWS BETWEEN 10 PRECEDING AND 1 PRECEDING
    ) AS r10_ppg
FROM boxscores.player_boxscores_traditional_v3
WHERE clean_name = 'luka doncic'
"""

pd.read_sql(query, engine).head()

,game_id_api,team_id,team_city,team_name,team_tricode,team_slug,player_id,first_name,family_name,name_initial,...,turnovers,fouls_personal,points,plus_minus_points,game_id,minutes_decimal,player_name,clean_name,ppg,r10_ppg
0,0022400007,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,5,2,31,2.0,0022400007,41.083332,Luka Dončić,luka doncic,NaN,NaN
1,0022400024,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,3,0,26,28.0,0022400024,29.616667,Luka Dončić,luka doncic,31.000000,31.000000
2,0022400056,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,3,4,37,16.0,0022400056,40.466667,Luka Dončić,luka doncic,28.500000,28.500000
3,0022400074,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,4,3,28,6.0,0022400074,36.283333,Luka Dončić,luka doncic,31.333333,31.333333
4,0022400095,1610612742,Dallas,Mavericks,DAL,mavericks,1629029,Luka,Dončić,L. Dončić,...,2,4,40,-12.0,0022400095,39.416668,Luka Dončić,luka doncic,30.500000,30.500000


In [30]:
pbs = pd.read_csv("player_boxscores_2025_26.csv")

# =========================================================
# 0) Sort for correct window behavior
# =========================================================

# Convert game_date to datetime 
pbs["game_date"] = pd.to_datetime(pbs["game_date"])

pbs = (
    pbs
    .sort_values(["personId", "game_date"])
    .reset_index(drop=True)
)

# =========================================================
# 1) Season-to-date average PPG (before current game)
# =========================================================

pbs["szn_avg_ppg"] = (
    pbs
    .groupby("personId")["points"]
    .transform(lambda s: s.shift(1).expanding().mean())
)

# =========================================================
# 2) Rolling 10-game average PPG (before current game)
# =========================================================

pbs["r10_avg_ppg"] = (
    pbs
    .groupby("personId")["points"]
    .transform(lambda s: s.shift(1).rolling(10, min_periods=1).mean())
)

# =========================================================
# 3) Sanity check (example player)
# =========================================================

luka = pbs[pbs["firstName"] == "Luka"]

luka[["game_id", "game_date", "points", "szn_avg_ppg", "r10_avg_ppg"]].head(15)


/var/folders/zb/kdjvw_5d73q1b2mg3fkcy8pw0000gn/T/ipykernel_39276/3336547417.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pbs["game_date"] = pd.to_datetime(pbs["game_date"])


,game_id,game_date,points,szn_avg_ppg,r10_avg_ppg
4245,22500002,2025-10-21,43,NaN,NaN
4246,22500019,2025-10-24,49,43.000000,43.000000
4247,22500024,2025-10-31,44,46.000000,46.000000
4248,22500155,2025-11-02,29,45.333333,45.333333
4249,22500179,2025-11-05,35,41.250000,41.250000
4250,22500185,2025-11-08,22,40.000000,40.000000
4251,22500198,2025-11-10,38,37.000000,37.000000
4252,22500222,2025-11-12,19,37.142857,37.142857
4253,22500045,2025-11-14,24,34.875000,34.875000
4254,22500231,2025-11-15,41,33.666667,33.666667


In [10]:
from nba_api.stats.endpoints import leaguegamelog
import pandas as pd
from pathlib import Path

# -----------------------------
# Config
# -----------------------------
SEASON = "2025-26"
SEASON_TYPE = "Regular Season"
OUTPUT_PATH = Path("league_gamelog_2025_26.csv")

# -----------------------------
# Fetch LeagueGameLog
# -----------------------------
def fetch_league_gamelog():
    print(f"🔍 Fetching LeagueGameLog for {SEASON}...")

    lg = leaguegamelog.LeagueGameLog(
        season=SEASON,
        season_type_all_star=SEASON_TYPE,
        timeout=60
    )

    df = lg.get_data_frames()[0]
    print(f"📊 Rows fetched: {len(df)}")

    return df


# -----------------------------
# Load existing CSV (if any)
# -----------------------------
def load_existing_csv(path):
    if path.exists():
        df_existing = pd.read_csv(path)
        print(f"📁 Existing CSV found: {len(df_existing)} rows")
        return df_existing
    else:
        print("📁 No existing CSV found — creating new file")
        return pd.DataFrame()


# -----------------------------
# Append new rows safely
# -----------------------------
def append_new_rows(df_new, df_existing):
    if df_existing.empty:
        return df_new

    key_cols = ["GAME_ID", "TEAM_ID"]

    existing_keys = set(
        zip(df_existing["GAME_ID"], df_existing["TEAM_ID"])
    )

    mask = [
        (gid, tid) not in existing_keys
        for gid, tid in zip(df_new["GAME_ID"], df_new["TEAM_ID"])
    ]

    df_filtered = df_new[mask]

    print(f"➕ New rows to append: {len(df_filtered)}")

    return pd.concat([df_existing, df_filtered], ignore_index=True)


# -----------------------------
# Main
# -----------------------------
def main():
    df_new = fetch_league_gamelog()
    df_existing = load_existing_csv(OUTPUT_PATH)

    df_final = append_new_rows(df_new, df_existing)

    OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(OUTPUT_PATH, index=False)

    print(f"✅ CSV saved → {OUTPUT_PATH}")
    print(f"📦 Total rows in file: {len(df_final)}")


if __name__ == "__main__":
    main()


🔍 Fetching LeagueGameLog for 2025-26...
📊 Rows fetched: 1070
📁 No existing CSV found — creating new file
✅ CSV saved → league_gamelog_2025_26.csv
📦 Total rows in file: 1070


In [13]:
tbs = pd.read_csv("league_gamelog_2025_26.csv")
tbs.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE'],
      dtype='object')

In [24]:
import pandas as pd

# =========================================================
# 0) Clean slate: drop existing derived columns if present
# =========================================================

cols_to_drop = [
    "pts_allowed",
    "opp_avg_pts_allowed",
    "opp_r10_pts_allowed",
]

tbs = tbs.drop(columns=[c for c in cols_to_drop if c in tbs.columns])

# =========================================================
# 1) Sort once for deterministic behavior
# =========================================================

tbs = (
    tbs
    .sort_values(["TEAM_ABBREVIATION", "GAME_DATE", "GAME_ID"])
    .reset_index(drop=True)
)

# =========================================================
# 2) Build opponent lookup (PTS → pts_allowed)
# =========================================================

opp = (
    tbs[["GAME_ID", "TEAM_ABBREVIATION", "PTS"]]
    .rename(columns={
        "TEAM_ABBREVIATION": "OPP_TEAM",
        "PTS": "pts_allowed"
    })
)

# =========================================================
# 3) Merge + keep opponent row only
# =========================================================

merged = tbs.merge(
    opp,
    on="GAME_ID",
    how="left",
    validate="many_to_many",
    suffixes=("", "_opp")  # safe now
)

merged = merged[merged["TEAM_ABBREVIATION"] != merged["OPP_TEAM"]]

tbs = merged.drop(columns="OPP_TEAM")

# =========================================================
# 4) Opponent season-to-date avg pts allowed (before game)
# =========================================================

tbs = (
    tbs
    .sort_values(["TEAM_ABBREVIATION", "GAME_DATE", "GAME_ID"])
    .reset_index(drop=True)
)

tbs["opp_avg_pts_allowed"] = (
    tbs
    .groupby("TEAM_ABBREVIATION")["pts_allowed"]
    .transform(lambda s: s.shift(1).expanding().mean())
)

# =========================================================
# 5) Opponent rolling 10-game avg pts allowed (before game)
# =========================================================

tbs["opp_r10_pts_allowed"] = (
    tbs
    .groupby("TEAM_ABBREVIATION")["pts_allowed"]
    .transform(lambda s: s.shift(1).rolling(10, min_periods=1).mean())
)

# =========================================================
# 6) Sanity check
# =========================================================

(
    tbs[tbs["TEAM_ABBREVIATION"] == "LAL"]
    [["GAME_ID", "GAME_DATE", "PTS", "pts_allowed", "opp_avg_pts_allowed", "opp_r10_pts_allowed"]]
    .head(15)
)


,GAME_ID,GAME_DATE,PTS,pts_allowed,opp_avg_pts_allowed,opp_r10_pts_allowed
464,22500002,2025-10-21,109,119,NaN,NaN
465,22500019,2025-10-24,128,110,119.000000,119.000000
466,22500113,2025-10-26,127,120,114.500000,114.500000
467,22500122,2025-10-27,108,122,116.333333,116.333333
468,22500136,2025-10-29,116,115,117.750000,117.750000
469,22500024,2025-10-31,117,112,117.200000,117.200000
470,22500155,2025-11-02,130,120,116.333333,116.333333
471,22500163,2025-11-03,123,115,116.857143,116.857143
472,22500179,2025-11-05,118,116,116.625000,116.625000
473,22500185,2025-11-08,102,122,116.555556,116.555556
